In [1]:
#imports
import os, sys, json  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import glob as glob

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown


from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file
# %%

# Script purpose
Inputs:
- leg ref
- 

Outputs:
com_info for selected state

In [ ]:
#pull in leg ref
leg_ref = pd.read_csv(get_recent_file("*.csv", r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\key_creation\2025"))
print(leg_ref.columns)
# leg_ref.head(2)


setup = leg_ref.loc[:,['state abbreviation', 'chamber']]
setup = setup.drop_duplicates()

In [4]:
def get_subset(chamber,state):
    state_leg_ref = leg_ref[(leg_ref['chamber'] == chamber) & (leg_ref['state abbreviation'] == state)].reset_index(drop = False)
    print(state_leg_ref.head().to_string())
    return state_leg_ref

## Pull in committee data files

In [21]:
committee_info_folder = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files'

In [ ]:
#testing get_subset
# com_ref = get_subset('House', 'MO')
# com_ref = com_ref.drop('index', axis = 1)
# com_ref

# Begin Assisgnment loop

In [ ]:
# state declaration
state = "VA"

In [ ]:
#get desired file
import re
os.chdir(committee_info_folder)
files = glob.glob('*.xlsx')
print(files)
state_abrv = state
for file in files:
    if re.search(rf'^{state_abrv}', str(file)):
        needed_file = file
        break

com_data = pd.read_excel(needed_file)
# com_data


['MO_senate.xlsx', 'ND_coms.xlsx', 'VA_house_comms.xlsx']
VA_house_comms.xlsx


In [ ]:
#checking names
for c in com_data.columns:
    if 'name' in c:
        needed_col = c
        print(needed_col)
unique_names = list(set(com_data.loc[:,f'{needed_col}'].to_list()))

more_unique = [x for x in unique_names if len(x.strip().split(' ')) > 2]
print(*more_unique, sep= "\n")


In [ ]:
def get_lname(name):
    # This pattern captures either a compound last name starting with a common prefix (La, Du, De)
    # or a single-word last name, ignoring optional suffixes.
    pattern = re.compile(
        r'(?P<last>(?:(?:La|Du|De)\s+\S+)|\S+)\s*(?:Jr\.?|Sr\.?|II|III|IV)?\s*$',
        re.UNICODE
    )
    match = pattern.search(name)
    if match:
        return match.group('last')
    else:
        print(f"No match for: {name}")
        return None

    

In [ ]:
#set up com ref

com_house = get_subset("House", state)
com_senate = get_subset("Senate", state)

com_ref = pd.concat([com_house, com_senate]).reset_index(drop=False)


   index     full_pk primary_key  district_code state abbreviation chamber                    title first name  last name       party  district  date assumed office                                  name  tenure leader  state_code  chamber_code
0    817  34001700.0      340017           17.0                 MO   House  Missouri Representative       Bill      Allen  Republican      17.0                 2022         MO Rep. Bill Allen (R-MO-017)       3    NaN        34.0           0.0
1    818  34011300.0      340113          113.0                 MO   House  Missouri Representative       Phil      Amato  Republican     113.0                 2022         MO Rep. Phil Amato (R-MO-113)       3    NaN        34.0           0.0
2    819  34007600.0      340076           76.0                 MO   House  Missouri Representative     Marlon   Anderson    Democrat      76.0                 2020    MO Rep. Marlon Anderson (D-MO-076)       5    NaN        34.0           0.0
3    820  34007100.0    

In [ ]:
#go through al rows and assign fpk

"""
This section will popup automatically
with a view of rows not assigned an fpk
"""

#iterate rows
com_data['full_pk'] = np.nan
for i, row in com_data.iterrows():
    # print(i)
    # print(row['name'])
    # print(row['chamber'])
    if row['chamber'] == 'Senate':
        ref = com_senate
    else:
        ref = com_house
    # print(type(row['district']))


    results = ref[ref['district_code'].astype(int) == row['district']]
    if len(results) > 1:
        print(results.head().to_string())
        print('ISSUE HERE')
        break
    full_pk = int(results['full_pk'].iloc[0])
    com_data.loc[i,'full_pk'] = full_pk

#check for empty values
com_data[com_data['full_pk'].isna()]

,chamber,com name,name,role,district,full_pk


In [ ]:

#final clean and export
com_data.insert(0, 'full_pk', com_data.pop('full_pk'))
com_data.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\full_pk_assigned\{state}_com_info.xlsx', index = False)


# Appendix

In [ ]:
# *testing* for logic to get last name
import re

names = [
    "Elgie R. Sims Jr.",
    'Elizabeth "Lisa" Hernandez',
    "Amy L. Grant",
    "Terra Costa Howard",
    "Julie A. Morrison",
    "Laura Faver Dias",
    "William E Hauter",
    "Aarón M. Ortíz",
    "Kimberly Du Buclet",
    "Suzanne M. Ness",
    "Andrew S. Chesney",
    "Janet Yang Rohr",
    "Mary Beth Canty",
    "Jason R. Bunting",
    "Michael J. Coffey Jr.",
    "Li Arellano Jr.",
    "Kimberly A. Lightford",
    "Joe C. Sosnowski",
    "Adam M. Niemerg",
    'William "Will" Davis',
    "Sally J. Turner",
    "Kevin John Olickal",
    "Camille Y. Lilly",
    "La Shawn K. Ford",
    "Marcus C. Evans Jr.",
    "Wayne A. Rosenthal",
    "Michael W. Halpin",
    "Christopher \"C.D.\" Davidsmeyer",
    "Darby A. Hills",
    "Nicole La Ha",
    "Tracy Katz Muhl"
]

# Regex pattern explanation:
#   - (?P<last>...): Captures the last name in a named group "last".
#   - (?:[A-Za-zÀ-ÖØ-öø-ÿ]+(?:\s+[A-Za-zÀ-ÖØ-öø-ÿ]+)*):
#         Matches one or more letters (including accented letters) possibly followed by additional words (for multi-word last names).
#   - \s*(?:Jr\.?|Sr\.?|II|III|IV)?\s*$:
#         Optionally matches common suffixes (Jr., Sr., II, III, IV) and any trailing whitespace until the end of the string.
pattern = re.compile(r'(?P<last>(?:[A-Za-zÀ-ÖØ-öø-ÿ]+(?:\s+[A-Za-zÀ-ÖØ-öø-ÿ]+)*))\s*(?:Jr\.?|Sr\.?|II|III|IV)?\s*$', re.UNICODE)

for name in names:
    match = pattern.search(name)
    if match:
        print(f"Name: {name} --> Last Name: {match.group('last')}")
    else:
        print(f"No match for: {name}")
